In [ ]:
import os
import sys
import time as t_util
import numpy as np
import cftime
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib
import mplotutils as mpu
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import yaml


In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:    path_main  = file.read()
with open('../path_grids.txt', 'r') as file:   path_grids = file.read()
    
dir_data_Ymax = f'{path_main}Data/EURO-CORDEX/Yearmax/'
dir_data_EMT  = f'{path_main}Data/EURO-CORDEX/EMT/'
dir_scripts   = f'{path_main}Scripts/'
dir_names     = f'{path_main}Scripts/Model_lists/'
dir_regrid    = f'{path_grids}Regridding/'


## Prepare variables and parameters

In [ ]:
#Define models
all_models = []
with open(dir_names + 'Models_CORDEX-EUR-11_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models.append(eval(line[:-1]))

#Define reference period
ref_period = '1981-2010'

#Define EMT
dEMT_str = 'ESAT-3.0K'
        
#Read standard grid
fname_grid_stand = dir_regrid + 'Standard_grid_CORDEX-EUR-11_NCC-NorESM1-M_SMHI-RCA4.nc'
grid_standard = xr.open_dataset(fname_grid_stand)


## Calculate change of TXx (fut - ref)

In [ ]:
variables = ['tasmin', 'tasmax']

vars_out = dict()
vars_out['tasmax'] = 'TXx'
vars_out['tasmin'] = 'TNx'

data_all = dict()

for variab in variables:

    dir_data = dir_data_Ymax + variab + '/'

    print(len(all_models), end=': ')

    #Loop over all models
    create = 1
    for i, model in enumerate(all_models):

        print(i, end=', ')

        #Get file names
        f_ref = [file for file in os.listdir(dir_data) if "_".join(model) in file and ref_period in file]
        f_app = [file for file in os.listdir(dir_data) if "_".join(model) in file and dEMT_str in file]
        if len(f_ref)!=1:  sys.exit('File is not unique')
        if len(f_app)!=1:  sys.exit('File is not unique')

        #Read data and calculate difference and rename
        data_ref = xr.open_dataset(dir_data + f_ref[0])
        data_app = xr.open_dataset(dir_data + f_app[0])
        data_diff = data_app.mean('time') - data_ref.mean('time')
        data_diff = data_diff.rename({variab: vars_out[variab]})
        
        #Save in NetCDF
        fname_tmp = dir_data_Ymax + "_".join(model) + '_' + variab + "_tmp.nc"
        data_diff.to_netcdf(fname_tmp)

        #Regrid to standard grid
        fname_grid = dir_regrid + 'grid_xy_CORDEX-EUR-11_NCC-NorESM1-M_SMHI-RCA4'
        fname_regrid = dir_data_Ymax + "_".join(model) + variab + "_regridded.nc"
        os.system("cdo remapbil," + fname_grid + " " + fname_tmp + " " + fname_regrid)

        #Read regridded dataset
        data_regr = xr.open_dataset(fname_regrid)
        data_regr = data_regr.reindex({'rlat': grid_standard['rlat'], 'rlon': grid_standard['rlon']}, method='nearest')

        #Remove unnecessary variables
        vars_remove = set(data_regr.data_vars).difference([vars_out[variab]])
        data_regr = data_regr.drop(vars_remove)
        if 'height' in data_regr: data_regr = data_regr.drop('height')

        #Collect in one array
        if create==1:
            data_coll = data_regr
            create = 0
        else:
            data_coll = xr.concat((data_coll, data_regr), dim='model')

        #Remove temporary files
        os.remove(fname_tmp)
        os.remove(fname_regrid)

    #Add model names
    data_coll['model'] = ["_".join(model) for model in all_models]
    
    #Save in file
    fname_out = dir_data_Ymax + vars_out[variab] + '_all_models_yearmax.nc'
    if os.path.exists(fname_out): os.remove(fname_out)
    data_coll.to_netcdf(fname_out)
    

## Calculate IQR of TXx in reference period (1981-2010)

In [ ]:
variables = ['tasmax']
vars_out = dict()
vars_out['tasmax'] = 'TXx'
vars_out['tasmin'] = 'TNx'

data_all = dict()

for variab in variables:

    dir_data = dir_data_Ymax + variab + '/'

    print(len(all_models), end=': ')

    #Loop over all models
    create = 1
    for i, model in enumerate(all_models):

        print(i, end=', ')

        #Get file names
        f_ref = [file for file in os.listdir(dir_data) if "_".join(model) in file and ref_period in file]
        if len(f_ref)!=1:  sys.exit('File is not unique')

        #Read data and calculate IQR
        data_read = xr.open_dataset(dir_data + f_ref[0])
        data_Q75  = data_read.quantile(0.75, dim='time')
        data_Q25  = data_read.quantile(0.25, dim='time')
        data_IQR  = data_Q75 - data_Q25
        
        #Rename variable
        data_IQR  = data_IQR.rename({variab: vars_out[variab]})
        
        #Save in NetCDF
        fname_tmp = dir_data_Ymax + "_".join(model) + '_' + variab + "_tmp_IQR_TXx.nc"
        if os.path.exists(fname_tmp): os.remove(fname_tmp)
        data_IQR.to_netcdf(fname_tmp)

        #Regrid to standard grid
        fname_grid = dir_regrid + 'grid_xy_CORDEX-EUR-11_NCC-NorESM1-M_SMHI-RCA4'
        fname_regrid = dir_data_Ymax + "_".join(model) + variab + "_regridded_tmp_IQR_TXx.nc"
        if os.path.exists(fname_regrid): os.remove(fname_regrid)
        os.system("cdo remapbil," + fname_grid + " " + fname_tmp + " " + fname_regrid)

        #Read regridded dataset
        data_regr = xr.open_dataset(fname_regrid)
        data_regr = data_regr.reindex({'rlat': grid_standard['rlat'], 'rlon': grid_standard['rlon']}, method='nearest')

        #Remove unnecessary variables
        vars_remove = set(data_regr.data_vars).difference([vars_out[variab]])
        data_regr = data_regr.drop(vars_remove)
        if 'height' in data_regr: data_regr = data_regr.drop('height')

        #Collect in one array
        if create==1:
            data_coll = data_regr
            create = 0
        else:
            data_coll = xr.concat((data_coll, data_regr), dim='model')

        #Remove temporary files
        os.remove(fname_tmp)
        os.remove(fname_regrid)
        
    #Save in file
    fname_save = dir_data_Ymax + vars_out[variab] + '-IQR_all_models.nc'
    if os.path.exists(fname_save): os.remove(fname_save)
    data_coll.to_netcdf(fname_save)
